# Historical Flood Data

In [1]:
import pandas as pd
# historical flood data
def clean_flood_data():
    flood_df = pd.read_excel('FloodArchive.xlsx', engine='openpyxl')

    #filter only for United States
    flood_df = flood_df[flood_df['Country'] == 'USA']
    #add a zip code based on the long and lat

    return flood_df

flood_df = clean_flood_data()

In [2]:
print(f'USA number of flood incidents: {len(flood_df)}')
print(flood_df.head())
#save as csv
flood_df.to_csv('united_states_floods.csv', index=False)


USA number of flood incidents: 477
    ID GlideNumber Country OtherCountry      long      lat       Area  \
8    9           0     USA            0  -85.1742  40.6691  210527.96   
10  11           0     USA            0  -89.5537  40.6814   26266.14   
11  12           0     USA            0 -108.0930  35.3824   26527.13   
12  13           0     USA            0  -96.7845  29.6044  141508.00   
13  14           0     USA            0  -83.5377  42.0122   16883.54   

        Began      Ended Validation  Dead  Displaced          MainCause  \
8  1985-02-22 1985-03-01       News     7       2250  Rain and snowmelt   
10 1985-03-03 1985-03-08       News     4       2400  Rain and snowmelt   
11 1985-03-13 1985-03-14       News     0         80  Rain and snowmelt   
12 1985-03-14 1985-03-15       News     0          0         Heavy rain   
13 1985-03-30 1985-03-31       News     0        300         Heavy rain   

    Severity  
8        2.0  
10       2.0  
11       1.0  
12       1.0  


In [3]:
# !pip install folium
import folium
import math

#visual representation of FLOOD DATA
map_usa = folium.Map(location=[39.8283, -98.5795], zoom_start=5)
severity_colors = {
    1.0: "yellow", #large flood events: 1-2 decades-long reported interval since the last similar event
    1.5: "orange", #very large events: greater than 2 decades but less than 100 year estimated recurrence interval
    2.0: "red" #Extreme events: with an estimated recurrence interval greater than 100 years.
}

# Add a marker for each flood occurrence
for index, row in flood_df.iterrows():
    # Add a circle marker at each flood location (lat, long)
    folium.CircleMarker(
        location=[row['lat'], row['long']],
        popup=f"ID: {row['ID']} | Severity: {row['Severity']} | Displaced: {row['Displaced']} | Date: {row['Began']}",
        color=severity_colors[row['Severity']] ,
        fill=True,
        fill_opacity=0.6
    ).add_to(map_usa)

# Save map to HTML file
map_usa.save('flood_map.html')


In [4]:
!pip install sqlalchemy_mate==2.0.0.0

#add zipcode column to flood data
from uszipcode import SearchEngine

search = SearchEngine()

def get_zipcode(lat, lon):
    result = search.by_coordinates(lat, lon)
    if result:
        return result[0].zipcode
    return None

# Apply the get_zipcode function to each row and create a new column 'zipcode'
flood_df['zipcode'] = flood_df.apply(lambda row: get_zipcode(row['lat'], row['long']), axis=1)


/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Download /root/.uszipcode/simple_db.sqlite from https://github.com/MacHu-GWU/uszipcode-project/releases/download/1.0.1.db/simple_db.sqlite ...
  1.00 MB downloaded ...
  2.00 MB downloaded ...
  3.00 MB downloaded ...
  4.00 MB downloaded ...
  5.00 MB downloaded ...
  6.00 MB downloaded ...
  7.00 MB downloaded ...
  8.00 MB downloaded ...
  9.00 MB downloaded ...
  10.00 MB downloaded ...
  11.00 MB downloaded ...
  Complete!


In [5]:
#save as csv
pd.set_option('display.max_columns', None)
print(flood_df.head())
flood_df.to_csv('united_states_floods.csv', index=False)

    ID GlideNumber Country OtherCountry      long      lat       Area  \
8    9           0     USA            0  -85.1742  40.6691  210527.96   
10  11           0     USA            0  -89.5537  40.6814   26266.14   
11  12           0     USA            0 -108.0930  35.3824   26527.13   
12  13           0     USA            0  -96.7845  29.6044  141508.00   
13  14           0     USA            0  -83.5377  42.0122   16883.54   

        Began      Ended Validation  Dead  Displaced          MainCause  \
8  1985-02-22 1985-03-01       News     7       2250  Rain and snowmelt   
10 1985-03-03 1985-03-08       News     4       2400  Rain and snowmelt   
11 1985-03-13 1985-03-14       News     0         80  Rain and snowmelt   
12 1985-03-14 1985-03-15       News     0          0         Heavy rain   
13 1985-03-30 1985-03-31       News     0        300         Heavy rain   

    Severity zipcode  
8        2.0   46781  
10       2.0   61611  
11       1.0   87045  
12       1.0   789

#Historical Housing Data

In [6]:
valid_zipcodes = flood_df[~flood_df['zipcode'].isna()]['zipcode'].unique()
def clean_housing_data():
    df = pd.read_csv('hpi_at_bdl_zip5.csv', dtype={'Five-Digit ZIP Code': str})
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    #zipcodes from flood data
    df = df[df['Five-Digit ZIP Code'].isin(valid_zipcodes)]
    return df

df_housing = clean_housing_data()

In [7]:
print(df_housing.head(10))
#save as csv
df_housing.to_csv('united_states_housing.csv', index=False)


     Five-Digit ZIP Code  Year  Annual Change (%)     HPI  HPI with 1990 base  \
2401               01253  2001                NaN  100.00                 NaN   
2402               01253  2002               9.38  109.38                 NaN   
2403               01253  2003               2.17  111.75                 NaN   
2404               01253  2004              21.80  136.12                 NaN   
2405               01253  2005               8.09  147.13                 NaN   
2406               01253  2006              12.92  166.14                 NaN   
2407               01253  2007              -2.73  161.60                 NaN   
2408               01253  2008              -1.61  159.00                 NaN   
2409               01253  2009              -2.66  154.77                 NaN   
2410               01253  2010               0.33  155.28                 NaN   

      HPI with 2000 base  
2401                 NaN  
2402                 NaN  
2403                 NaN  


#Random Forest Regressor

In [8]:
#code for predicing HPI based on flood data
#load data
import pandas as pd
flood_data = pd.read_csv('united_states_floods.csv', dtype={'zipcode': str})
housing_data = pd.read_csv('united_states_housing.csv', dtype={'Five-Digit ZIP Code': str})
print(flood_data.head(5))


   ID GlideNumber Country OtherCountry      long      lat       Area  \
0   9           0     USA            0  -85.1742  40.6691  210527.96   
1  11           0     USA            0  -89.5537  40.6814   26266.14   
2  12           0     USA            0 -108.0930  35.3824   26527.13   
3  13           0     USA            0  -96.7845  29.6044  141508.00   
4  14           0     USA            0  -83.5377  42.0122   16883.54   

        Began       Ended Validation  Dead  Displaced          MainCause  \
0  1985-02-22  1985-03-01       News     7       2250  Rain and snowmelt   
1  1985-03-03  1985-03-08       News     4       2400  Rain and snowmelt   
2  1985-03-13  1985-03-14       News     0         80  Rain and snowmelt   
3  1985-03-14  1985-03-15       News     0          0         Heavy rain   
4  1985-03-30  1985-03-31       News     0        300         Heavy rain   

   Severity zipcode  
0       2.0   46781  
1       2.0   61611  
2       1.0   87045  
3       1.0   78962  


In [9]:
print(housing_data.head(5))

  Five-Digit ZIP Code  Year  Annual Change (%)     HPI  HPI with 1990 base  \
0               01253  2001                NaN  100.00                 NaN   
1               01253  2002               9.38  109.38                 NaN   
2               01253  2003               2.17  111.75                 NaN   
3               01253  2004              21.80  136.12                 NaN   
4               01253  2005               8.09  147.13                 NaN   

   HPI with 2000 base  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  


In [10]:
#SUMMARIZE FLOOD EVENTS
# For each zip code, get a summary of the floods that occurred up to and including that year.
# calculate duration(in days)
flood_data['Began'] = pd.to_datetime(flood_data['Began'])
flood_data['Ended'] = pd.to_datetime(flood_data['Ended'])
flood_data.loc[:, 'Duration'] = (flood_data['Ended'] - flood_data['Began']).dt.days
flood_data['Year'] = flood_data['Began'].dt.year


# # aggregate flood data by zip code and year
flood_summary = flood_data.groupby(['zipcode', 'Year']).agg(
    flood_count=('Severity', 'count'),
    max_severity=('Severity', lambda x: x.max()),
    median_dead=('Dead', 'median'),
    median_displaced=('Displaced', 'median'),
    median_duration=('Duration', 'median'),
    median_area = ('Area', 'median')
).reset_index()

flood_summary



,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area
0,00641,1985,1,1.0,2.0,4500.0,1.0,1423.92
1,00664,1985,1,1.0,99.0,6200.0,2.0,7219.80
2,00698,2001,1,1.0,2.0,0.0,1.0,245.95
3,01031,1996,1,2.0,5.0,700.0,4.0,67013.98
4,01031,2007,1,1.0,9.0,0.0,6.0,109079.09
...,...,...,...,...,...,...,...,...
450,99705,1991,1,1.0,2.0,800.0,4.0,22130.06
451,99705,2003,1,1.0,0.0,0.0,4.0,1681.67
452,99709,2003,1,1.0,0.0,0.0,5.0,6718.98
453,99743,2003,1,1.0,0.0,20.0,4.0,13111.55


In [13]:
housing_data.rename(columns={'Five-Digit ZIP Code': 'zipcode'}, inplace=True)

# Merge flood summary with housing price index data based on 'zipcode' and 'Year'
merged_data = pd.merge(flood_summary, housing_data, on=['zipcode', 'Year'], how='left')

# Display the first few rows of the merged data
merged_data

,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,00641,1985,1,1.0,2.0,4500.0,1.0,1423.92,NaN,NaN,NaN,NaN
1,00664,1985,1,1.0,99.0,6200.0,2.0,7219.80,NaN,NaN,NaN,NaN
2,00698,2001,1,1.0,2.0,0.0,1.0,245.95,NaN,NaN,NaN,NaN
3,01031,1996,1,2.0,5.0,700.0,4.0,67013.98,NaN,NaN,NaN,NaN
4,01031,2007,1,1.0,9.0,0.0,6.0,109079.09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
450,99705,1991,1,1.0,2.0,800.0,4.0,22130.06,20.76,72.44,120.76,69.72
451,99705,2003,1,1.0,0.0,0.0,4.0,1681.67,4.43,113.15,188.63,108.90
452,99709,2003,1,1.0,0.0,0.0,5.0,6718.98,2.88,246.91,162.72,110.30
453,99743,2003,1,1.0,0.0,20.0,4.0,13111.55,NaN,NaN,NaN,NaN


In [17]:
#drop unmatched rows
#NOTE ROWS DROPPED PRETTY SIGNIFICANTLY
merged_data = merged_data.dropna()
merged_data

,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
6,01749,2018,1,1.5,0.0,0.0,2.0,5095.45,6.20,468.17,230.13,164.19
12,03903,2006,1,2.0,1.0,2500.0,11.0,42910.99,3.83,314.67,214.37,164.46
20,07753,2011,1,2.0,20.0,370000.0,17.0,252912.89,-7.22,562.36,194.21,161.89
22,08088,2004,1,2.0,0.0,500.0,3.0,2441.81,14.87,395.73,163.53,144.36
32,17036,2018,1,1.5,1.0,0.0,1.0,25377.62,4.30,265.31,187.85,152.03
...,...,...,...,...,...,...,...,...,...,...,...,...
442,98503,2008,1,1.5,2.0,0.0,6.0,75156.68,-1.42,578.90,289.04,181.96
443,98506,2007,1,1.0,2.0,1100.0,4.0,41684.03,4.41,653.48,320.03,188.83
450,99705,1991,1,1.0,2.0,800.0,4.0,22130.06,20.76,72.44,120.76,69.72
451,99705,2003,1,1.0,0.0,0.0,4.0,1681.67,4.43,113.15,188.63,108.90


In [18]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score


#Use RandomForestRegressor to predict HPI based on floods

X = merged_data[['flood_count', 'max_severity', 'median_dead', 'median_displaced', 'median_duration', 'median_area']]
y = merged_data['HPI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

# evaluate the model performance
r2 = r2_score(y_test, y_pred)
print(f'R² score: {r2:.3f}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae:.3f}')

print(f"Predictions: {y_pred[:10]}")
print(f"Actual: {y_test[:10].values}")

R² score: -0.707
Mean Absolute Error (MAE): 167.823
Predictions: [245.7141 278.2456 384.5934 364.4457 326.7193 148.5004 654.1843 298.0053
 294.3737 352.6352]
Actual: [508.46 236.46 160.39 171.49 348.77 188.54 286.93 194.88 242.57 468.17]
